In [2]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.0 MB/s eta 0:00:00


In [3]:
import json
import openai

# E-Mails aus JSON-Datei laden
with open('emails.json', 'r') as file:
    emails = json.load(file)

# OpenAI API-Client einrichten
openai.api_key = 'DEIN_API_SCHLÜSSEL'

# Vorlagen für Zusammenfassung und Klassifizierung definieren
summarize_prompt_template = "Betreff: {subject}\nAbsender E-Mail: {sender_email}\nZeit und Datum: {time_date}\nNachricht: {message}\n\nFasse die E-Mail in Stichpunkten zusammen:\n-"

classification_prompt_template = "Betreff: {subject}\nAbsender E-Mail: {sender_email}\nZeit und Datum: {time_date}\nNachricht: {message}\n\nKlassifiziere diese E-Mail in folgende Kategorien:\n1. Objektanfragen\n2. Verfügbarkeit und Preise\n3. Terminvereinbarung und Besichtigung\n4. Lärmbeschwerden\n5. Informationsanfragen\n6. Sonstige Beschwerden\n\nVertrauensscore: [0-1]"

# E-Mails verarbeiten und Anfragen an ChatGPT-API senden
for email in emails:
    # E-Mail zusammenfassen
    summarize_prompt = summarize_prompt_template.format(
        subject=email['subject'],
        sender_email=email['sender_email'],
        time_date=email['time_date'],
        message=email['message']
    )

    # Anfrage an ChatGPT-API für Zusammenfassung senden
    summarize_response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=summarize_prompt,
        max_tokens=100  # Je nach gewünschter Länge der Zusammenfassung anpassen
    )

    # API-Antwort für Zusammenfassung analysieren
    email['summary'] = summarize_response.choices[0].text.strip()

    # Aufgaben aus der Zusammenfassung generieren
    tasks = email['summary'].split('-')

    # Aufgaben bereinigen und leere Einträge entfernen
    tasks = [task.strip() for task in tasks if task.strip()]

    # Generierte Aufgaben der E-Mail zuordnen
    email['tasks'] = tasks

    # E-Mail klassifizieren
    classification_prompt = classification_prompt_template.format(
        subject=email['subject'],
        sender_email=email['sender_email'],
        time_date=email['time_date'],
        message=email['message']
    )

    # Anfrage an ChatGPT-API für Klassifizierung senden
    classification_response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=classification_prompt,
        max_tokens=1  # Je nach gewünschter Länge des Vertrauensscores anpassen
    )

    # API-Antwort für Klassifizierung analysieren
    predicted_category = classification_response.choices[0].text.strip()
    confidence_score = classification_response.choices[0].confidence

    # Vorhergesagte Kategorie und Vertrauensscore der E-Mail zuordnen
    email['predicted_category'] = predicted_category
    email['confidence_score'] = confidence_score

# Triagierte E-Mails speichern
with open('triaged_emails.json', 'w') as file:
    json.dump(emails, file, indent=4)


FileNotFoundError: [Errno 2] No such file or directory: 'emails.json'